In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)
import pandas as pd
import numpy as np

from elastic_helpers import ESSearch
from utils import InputProcessor, convert_to_excel_name

In [2]:
load_dotenv()
print(find_dotenv())
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

/home/a543979/hdegis-chat-backend/.env
/home/a543979/hdegis-chat-backend/key/pjt-dev-hdegis-app-454401-bd4fac2d452b.json


In [3]:
ES_HOST = "https://node.hd-aic.com:30692"
ES_USER = "genai_hde"
ES_PASSWORD = "wpsAI1@"

In [4]:
es = ESSearch(
    hosts=ES_HOST,
    credentials=(ES_USER, ES_PASSWORD)
)

df = pd.read_excel("QA 성능테스트 v4.xlsx")

/opt/miniforge3/envs/venv_chat_backend_test/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://node.hd-aic.com:30692' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/opt/miniforge3/envs/venv_chat_backend_test/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [22]:
# INDEX_NAME = "hde_hvcb_text_004"
# EMBEDDING_MODEL_NAME = "text-embedding-004"
# INDEX_NAME = "hde_hvcb_text_0815"
# EMBEDDING_MODEL_NAME = "text-embedding-005"
INDEX_NAME = "hde_hvcb_text_m_002"
EMBEDDING_MODEL_NAME = "text-multilingual-embedding-002"

TOP_K = 10
TEXT_FIELDS = ["content"]
VECTOR_FIELD = "embedding"

### Tolerance 포함 X

In [26]:
hit_at_k = []
reciprocal_ranks = []

for i, row in df.iterrows():
    ################### 사용자 입력 ###################
    user_query = row['query']
    user_filter = row['폴더 선택 (검색 범위 지정)']
    print(f"[{(i+1):02d}] {user_query} (Filter: {user_filter})")
    print("=================================")

    ################### 검색 ###################
    # response = es.vector_search_w_filter(
    #     index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     vector_field=VECTOR_FIELD,
    # )

    # response = es.keyword_text_search_w_filter(
    #     index_name=INDEX_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     text_fields=TEXT_FIELDS,
    # )
    
    # response = es.hybrid_search_w_filter(
    #     index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     vector_field=VECTOR_FIELD, text_fields=TEXT_FIELDS,
    # )

    # response = es.hyde_vector_search_w_filter(
    #     index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     vector_field=VECTOR_FIELD,
    # )

    response = es.hyde_hybrid_search_w_filter(
        index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
        user_query=user_query, user_filter=user_filter,
        vector_field=VECTOR_FIELD, text_fields=TEXT_FIELDS
    )

    hits = response['hits']['hits']

    ################### 검색결과 취합 ###################
    search_ref = []
    for idx, hit in enumerate(hits):
        folder_levels = convert_to_excel_name('/'.join(hit['_source']['folder_levels']))
        pdf_name = hit['_source']['pdf_name']
        page = int(hit['_source']['page'])
        score = hit['_score']
        search_ref.append((folder_levels, pdf_name, page))
        
    print("-----------------------------------")
    print("Search ref:")
    for s in search_ref:
        print(s)
    print("-----------------------------------")

    ################### 정답 취합 ###################
    gt_ref = []
    for n in range(1, 14):
        if pd.isna(row[f"page_{n}"]):
            break
        path = row[f"path_{n}"]
        filename = row[f"filename_{n}"]
        page = int(row[f'page_{n}'])
        gt_ref.append((path, filename, page))
    print("GT ref:")
    for g in gt_ref:
        print(g)
    print("-----------------------------------")

    ################### Hit Rate ###################
    hit = any(item in gt_ref for item in search_ref)
    hit_at_k.append(hit)
    print(f"Hit@K: {hit}")

    ################### Reciprocal Rank ###################
    reciprocal_rank = 0
    for rank, search_item in enumerate(search_ref, start=1):
        if search_item in gt_ref:
            reciprocal_rank = 1 / rank
            break
    reciprocal_ranks.append(reciprocal_rank)
    print(f"rr: {reciprocal_rank}")
    print("=================================")
    
    break

mrr = np.mean(reciprocal_ranks)
hit_at_k_score = np.mean(hit_at_k)
print(f"<< INDEX_NAME: {INDEX_NAME} | EMBEDDING_MODEL: {EMBEDDING_MODEL_NAME} | TOP_K: {TOP_K} >>")
print(f"Hit@{TOP_K}: {hit_at_k_score * 100:.2f}%")
print(f"Mean Reciporcal Rank (MRR): {mrr * 100:.2f}%")

[01] Are there any requirements regarding the operating method of the circuit breaker, such as spring-operated or hydraulic-operated? (Filter: 3. Customer Standard Specifications/Spain/REE)
Create HyDE document !
Create Query Vector for HyDE!
Create Keywords: ['circuit breaker', 'circuit breakers', 'operating method', 'operation', 'spring operated', 'spring-operated', 'hydraulic operated', 'hydraulic-operated', 'electrical switch', 'switchgear', 'control mechanism', 'tripping mechanism', 'power system', 'power distribution', 'standards', 'requirements', 'regulations', 'specifications', 'iec', 'ansi', 'ieee', 'safety', 'compliance']
Apply filter ! 
-----------------------------------
Search ref:
('Customer_Standard_Specifications__Spain__REE', 'ET164_ed4_English_No_Official.pdf', 20)
('Customer_Standard_Specifications__Spain__REE', 'ET164_ed4_English_No_Official.pdf', 44)
('Customer_Standard_Specifications__Spain__REE', 'ET082_ed8_Eng_NO_OFFICIAL.pdf', 7)
('Customer_Standard_Specificati

### Tolerance 추가버전

In [24]:
def is_hit_with_tolerance(search_item, gt_ref, tolerance):
    """Tolerance를 고려해서 search_item이 gt_ref 안에 있는지 검사"""
    search_path, search_filename, search_page = search_item
    for gt_path, gt_filename, gt_page in gt_ref:
        if (search_path == gt_path) and (search_filename == gt_filename):
            if abs(search_page - gt_page) <= tolerance:
                return True
    return False

In [25]:
def is_hit_with_tolerance(search_item, gt_ref, tolerance):
    """Tolerance를 고려해서 search_item이 gt_ref 안에 있는지 검사"""
    search_path, search_filename, search_page = search_item
    for gt_path, gt_filename, gt_page in gt_ref:
        if (search_path == gt_path) and (search_filename == gt_filename):
            if abs(search_page - gt_page) <= tolerance:
                return True
    return False

# tolerances: 정확 일치(0) 포함!
tolerances = [0, 1, 3, 5]
hit_at_ks = {tol: [] for tol in tolerances}
reciprocal_ranks_tols = {tol: [] for tol in tolerances}

for i, row in df.iterrows():
    ################ 사용자 입력 ################
    user_query = row['query']
    user_filter = row['폴더 선택 (검색 범위 지정)']
    # print(f"[{(i+1):02d}] {user_query} (Filter: {user_filter})")
    # print("=================================")

    ################ 검색 ################
    # response = es.vector_search_w_filter(
    #     index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     vector_field=VECTOR_FIELD,
    # )

    # response = es.keyword_text_search_w_filter(
    #     index_name=INDEX_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     text_fields=TEXT_FIELDS,
    # )
    
    # response = es.hybrid_search_w_filter(
    #     index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     vector_field=VECTOR_FIELD, text_fields=TEXT_FIELDS,
    # )

    # response = es.hyde_vector_search_w_filter(
    #     index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
    #     user_query=user_query, user_filter=user_filter,
    #     vector_field=VECTOR_FIELD,
    # )

    response = es.hyde_hybrid_search_w_filter(
        index_name=INDEX_NAME, embedding_model_name=EMBEDDING_MODEL_NAME, top_k=TOP_K,
        user_query=user_query, user_filter=user_filter,
        vector_field=VECTOR_FIELD, text_fields=TEXT_FIELDS
    )

    hits = response['hits']['hits']

    ################ 검색결과 정리 ################
    search_ref = []
    for hit in hits:
        folder_levels = convert_to_excel_name('/'.join(hit['_source']['folder_levels']))
        pdf_name = hit['_source']['pdf_name']
        page = int(hit['_source']['page'])
        search_ref.append((folder_levels, pdf_name, page))
    
    ################ 정답 정리 ################
    gt_ref = []
    for n in range(1, 14):
        if pd.isna(row[f"page_{n}"]):
            break
        path = row[f"path_{n}"]
        filename = row[f"filename_{n}"]
        page = int(row[f'page_{n}'])
        gt_ref.append((path, filename, page))

    ################ Hit@K, MRR 계산 ################
    for tol in tolerances:
        # Hit@K
        hit = any((search_item, gt_ref, tol) for search_item in search_ref)
        hit_at_ks[tol].append(hit)

        # Reciprocal Rank
        reciprocal_rank = 0
        for rank, search_item in enumerate(search_ref, start=1):
            if is_hit_with_tolerance(search_item, gt_ref, tol):
                reciprocal_rank = 1 / rank
                break
        reciprocal_ranks_tols[tol].append(reciprocal_rank)

    # print("-----------------------------------")
    # print("완료")
    # print("=================================")
    
    # break  # <- 이거 풀어야 전체 데이터에 대해 돌릴 수 있음

################ 최종 결과 출력 ################
print(f"<< INDEX_NAME: {INDEX_NAME} | EMBEDDING_MODEL: {EMBEDDING_MODEL_NAME} | TOP_K: {TOP_K} >>")
for tol in tolerances:
    mrr = np.mean(reciprocal_ranks_tols[tol])
    hit_at_k_score = np.mean(hit_at_ks[tol])
    if tol == 0:
        print(f"[정확 일치]")
    else:
        print(f"[Tolerance ±{tol}]")
    print(f"Hit@{TOP_K}: {hit_at_k_score * 100:.2f}%")
    print(f"Mean Reciprocal Rank (MRR): {mrr * 100:.2f}%")
    print("--------------------------------------------")


Create HyDE document !
Create Query Vector for HyDE!
Create Keywords: ['circuit breaker', 'circuit breakers', 'operating method', 'operation', 'spring-operated', 'spring operated', 'hydraulic-operated', 'hydraulic operated', 'electrical switchgear', 'switchgear', 'switch gear', 'mechanism', 'requirements', 'standard', 'regulation', 'specifications', 'IEC', 'ANSI', 'IEEE', 'tripping', 'trip', 'close', 'closing', 'open', 'opening', 'manual operation', 'remote operation', 'automatic operation', 'control', 'controlled']
Apply filter ! 
Create HyDE document !
Create Query Vector for HyDE!
Create Keywords: ['local control cubicle', 'LCC', 'standalone', 'integral', 'type', 'local', 'control', 'cubicle', 'standalone type', 'integral type', 'independent', 'separate', 'attached', 'built-in', 'integrated', 'self-contained', 'unitary']
Apply filter ! 
Create HyDE document !
Create Query Vector for HyDE!
Create Keywords: ['Chinese products', 'foreign products', 'imported products', 'domestic produc

In [28]:
hit_at_ks

{0: [False,
  False,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  False,
  False],
 1: [False,
  True,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  True,
  False,
  False],
 3: [True,
  True,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  True,
  True,
  True],
 5: [True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  True,
  True,
  True]}